In [ ]:
'dahyun+darwin = dahwin'

In [ ]:
!pip install transformers accelerate

In [ ]:
from transformers import GPT2LMHeadModel,GPT2Tokenizer
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import torch

In [ ]:
# from transformers import GPT2LMHeadModel, GPT2Tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2LMHeadModel.from_pretrained('gpt2')
# print(model.generate(tokenizer('I love dahyun',return_tensors='pt')))

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
input_text = "I love to eat"
input_ids = tokenizer.encode(input_text, return_tensors='pt')
output = model.generate(input_ids)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


In [ ]:
import time 
s = time.time()
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
input_text = "I love to eat"
input_ids = tokenizer.encode(input_text, return_tensors='pt')
output = model.generate(
    input_ids,
    max_length=100,
    num_return_sequences=3,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.9
)
generated_text = []
for seq in output:
    generated_text.append(tokenizer.decode(seq, skip_special_tokens=True))
    
generated_text_cleaned = []
for text in generated_text:
    cleaned_text = text.replace('\n', '').replace('\r', '').replace('\t', '')
    generated_text_cleaned.append(cleaned_text)
    
print(generated_text_cleaned)

e = time.time()
print(e-s)


In [ ]:
import time 
s = time.time()
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
input_text = "I love to eat"
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
input_ids = input_ids.to(device)
output = model.generate(
    input_ids,
    max_length=100,
    num_return_sequences=3,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.9
)

generated_text = []
for seq in output:
    generated_text.append(tokenizer.decode(seq, skip_special_tokens=True))
    
generated_text_cleaned = []
for text in generated_text:
    cleaned_text = text.replace('\n', '').replace('\r', '').replace('\t', '')
    generated_text_cleaned.append(cleaned_text)
    
print(generated_text_cleaned)
e = time.time()
print(e-s)

In [ ]:
from torch.utils.data import Dataset
import json

class ChatData(Dataset):
    def __init__(self, path:str, tokenizer):
        self.data = json.load(open(path, "r"))

        self.X = []
        for i in self.data:
            for j in i['dialog']:
                self.X.append(j['text'])

        for idx, i in enumerate(self.X):
            try:
                self.X[idx] = "<startofstring> "+i+" <bot>: "+self.X[idx+1]+" <endofstring>"
            except:
                break

        self.X = self.X[:5000]
        
        print(self.X[0])

        self.X_encoded = tokenizer(self.X,max_length=40, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [ ]:
tokenizer.add_special_tokens({"pad_token": "<pad>", 
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])
chatdata = ChatData('chat_data.json',tokenizer)


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import torch

def train(chatData, model, optim):

    epochs = 12

    for i in tqdm.tqdm(range(epochs)):
        for X, a in chatData:
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            loss = model(X, attention_mask=a, labels=X).loss
            loss.backward()
            optim.step()
        torch.save(model.state_dict(), "model_state.pt")
        print(infer("hello how are you"))

def infer(inp):
    inp = "<startofstring> "+inp+" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a )
    output = tokenizer.decode(output[0])
    return output


device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>", 
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model = model.to(device)

# print(tokenizer.decode(model.generate(**tokenizer("hey i was good at basketball but ",
#                          return_tensors="pt"))[0]))

chatData = ChatData("chat_data.json", tokenizer)
chatData =  DataLoader(chatData, batch_size=64)

model.train()

optim = Adam(model.parameters(), lr=1e-3)

print("training .... ")
train(chatData, model, optim)

print("infer from model : ")
while True:
  inp = input()
  print(infer(inp))

In [ ]:
# <startofstring> I Love Dahyun! More than my life! She is my girlfriend! <bot>: It's great to hear 
# that you love your girlfriend Dahyun so much! It's important to cherish and appreciate our loved ones.<endofstring>


In [ ]:
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import torch

def train(chatData, model, optim):

    epochs = 12

    for i in tqdm.tqdm(range(epochs)):
        for X, a in chatData:
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            loss = model(X, attention_mask=a, labels=X).loss
            loss.backward()
            optim.step()
        model.save_pretrained("model")
        print(infer("hello how are you"))

def infer(inp):
    inp = "<startofstring> "+inp+" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a )
    output = tokenizer.decode(output[0])
    return output


device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>", 
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model = model.to(device)

# print(tokenizer.decode(model.generate(**tokenizer("hey i was good at basketball but ",
#                          return_tensors="pt"))[0]))

chatData = ChatData("chat_data.json", tokenizer)
chatData =  DataLoader(chatData, batch_size=64)

model.train()

optim = Adam(model.parameters(), lr=1e-3)

print("training .... ")
train(chatData, model, optim)

print("infer from model : ")
while True:
  inp = input()
  print(infer(inp))


In [ ]:
import os

# replace with your file path
file_path = "model/pytorch_model.bin"

# get size of file in bytes
size_bytes = os.path.getsize(file_path)

# convert to megabytes
size_mb = size_bytes / (1024 * 1024)

print(f"File size: {size_mb:.2f} MB")


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from transformers import GPT2Config
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model_config_path = "model/config.json"
model_weights_path = "model/pytorch_model.bin"

model_config = GPT2Config.from_json_file(model_config_path)
model = GPT2LMHeadModel(config=model_config)
model.load_state_dict(torch.load(model_weights_path, map_location=torch.device('cpu')))

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def generate_conversation(inp):
    inp = "<startofstring> "+inp+" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    input_ids = inp["input_ids"].to(device)
    attention_mask = inp["attention_mask"].to(device)
    output = model.generate(input_ids=input_ids, attention_mask=attention_mask)
    output = output.flatten().tolist()
    output = [t for t in output if t is not None and tokenizer.decode([t]).strip() not in tokenizer.all_special_tokens]
    return tokenizer.decode(output, skip_special_tokens=True)


print(generate_conversation("what you like to do?"))

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from transformers import GPT2Config

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model_config_path = "model/config.json"
model_weights_path = "model/pytorch_model.bin"

model_config = GPT2Config.from_json_file(model_config_path)
model = GPT2LMHeadModel(config=model_config)
model.load_state_dict(torch.load(model_weights_path, map_location=torch.device('cpu')))

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def generate_conversation(prompt, max_length):
    prompt = "<startofstring> " + prompt + ": "
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    mask = [1 for _ in range(max_length)] * len(input_ids)
    input_ids = input_ids[:max_length - 1].unsqueeze(-1)  # remove eos token & pad
    attention_mask = torch.cat((1 - torch.tensor(mask)[:max_length - 1], torch.zeros(len(input_ids)).bool()), dim=-1).float().unsqueeze(-1)

    generated = ""
    while True:
        all_outs = []
        try:
            logits, _, _ = model(input_ids, attention_mask=attention_mask)
            predictions = logits[:, :-1, :]
            top_preds, top_scores = [], []
            for pred_index in range(len(predictions)):
                score = abs(predictions[:, pred_index, :])
                top_preds.append(list(range(num_choices)[::-1][score > thresh]))
                top_scores.append(max(score[-2:][top_preds[pred_index]]))
            chosen_pred = top_preds[0][sample(*top_scores[0])]
            generated += tokenizer.convert_ids_to_str([generated[:-1]] + [next(sorted(set(predicted_toks)), key=(lambda x: random()))])
            tokenized = tokenizer.batch_encode([generated])[0]

            input_ids = tokenized[:max_length - 1]
            attention_mask = torch.tensor([tokenizer.convert_tokens_to_ids(gen) == tokenizer.convert_tokens_to_ids(encoded_prompts) for gen in generated for encoded_prompts in generated[:generations]*2+1], dtype=torch.int64)
            attention_mask = attention_mask / 2**len(input_ids)*2 - 0.9
        except RuntimeError as e:
            break
    return generated


print(generate_conversation("what you like to do?", max_length=50))



In [ ]:
def generate_conversation(prompt, max_length):
    ...
    # Generate responses up to the specified max_length or until the end-of-sentence token
    response = []
    while len(response) < max_length and '<unk>' not in input_ids:
        batched_inputs, _ = model. generator(input_ids=input_ids[:max_length], attention_mask=attention_mask[:max_length])
        response += batched_inputs[:, :, -1].numpy()
        input_ids, attention_mask = zip(*batched_inputs)

    ...